<a href="https://colab.research.google.com/github/SamuelAmrich/TLEIA-TLEOS/blob/master/ML_Left_Cerberus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Príprava mašiny**



In [ ]:
#@title Default title text
# Nainstalovanie potrebnych dodatocnych kniznic

!pip3 install --upgrade pip

!pip3 install tensorflow
!pip3 install keras
!pip3 install opencv-python

!pip install split-folders
!pip install -q pyyaml h5py

     |████████████████████████████████| 1.5MB 3.5MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1


In [ ]:
# Importovanie kniznic

# Zakladne kniznice pre pracu
import math, time, os, random, shutil

# Pre dodatocne spracovanie 
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import PIL.Image
from PIL import Image
import cv2
from google.colab import drive

mpl.rcParams['figure.figsize'] = (12, 5)

# Pre neuronovu siet
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras import layers
AUTOTUNE = tf.data.experimental.AUTOTUNE

import keras

Using TensorFlow backend.


In [ ]:
# Nadefinovanie potrebnej plotovacej funkcie 
#Definicia pre vyplotenie jedneho obrazku

def plotImage(img):
    fig, axes = plt.subplots(1, 1, figsize=(5,5))
    axes.imshow(img[0])
    axes.axis('off')
    plt.tight_layout()
    plt.show()
    
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
# Namapuje google drive

!mkdir raw_data 
drive.mount('raw_data/')
path = '/content/raw_data/My Drive/Space::Lab/ML_Left/'
print("Cesta k RAW datam je: " + path)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at raw_data/
Cesta k RAW datam je: /content/raw_data/My Drive/Space::Lab/ML_Left/


**Nacitanie zoznamu a preorganizovanie obrazkov**

**Praca na obrazkoch**

In [ ]:
# A tereaz pekne odznovu

train_dir = "/content/raw_data/My Drive/Space::Lab/ML_Left/old/train"
validation_dir = "/content/raw_data/My Drive/Space::Lab/ML_Left/old/validation"


# Poscita pocet obrazkov

num_nothing_tr    =   len(os.listdir(train_dir+"/nothing"))
num_event_tr      =   len(os.listdir(train_dir+"/event"))

num_nothing_val   =   len(os.listdir(validation_dir+"/nothing"))
num_event_val     =   len(os.listdir(validation_dir+"/event"))

total_train       =   num_nothing_tr + num_event_tr
total_val         =   num_nothing_val + num_event_val

# Vypise ciselne pocty event-ov
print("--")
print('total training nothing images:', num_nothing_tr)
print('total training event images:', num_event_tr)
print("--")
print('total validation nothing images:', num_nothing_val)
print('total validation event images:', num_event_val)
print("--")
print("Total training images:", total_train)
print("Total validation images:", total_val)


#Nastavuje vlastnosti obrazkov a modelu

batch_size    =   55
epochs        =   10
IMG_HEIGHT    =   200
IMG_WIDTH     =   250
# Moje rozlisenie 250 x 200  /// Povodne rozlisenie 150 x 150

# Augmentacie obrazkov

image_gen_train = ImageDataGenerator(
                    rescale               =   1./255,       #preskalovanie
                    rotation_range        =   0,            #rotacia okolo stredu
                    width_shift_range     =   0.0,         #posunutie/roztiahnutie do sirky 
                    height_shift_range    =   0.0,         #posunutie/roztiahnutie do vysky
                    horizontal_flip       =   True,         #preklopenie obrazka okolo |
                    zoom_range            =   0.0,          #priblizenie obrazu
                    )

image_gen_train = ImageDataGenerator(rescale=1./255)
image_gen_val = ImageDataGenerator(rescale=1./255)

# Nacitanie trenovacich obrazkov do modelu

train_data_gen = image_gen_train.flow_from_directory(batch_size=batch_size,
                                                     directory=train_dir,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     class_mode='binary',
                                                     #color_mode='grayscale'
                                                     )


# Vygenreuje validovacie obrazky pre neuronku

val_data_gen = image_gen_val.flow_from_directory(batch_size=batch_size,
                                                 directory=validation_dir,
                                                 target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                 class_mode='binary',
                                                 #color_mode='grayscale'
                                                 )

--
total training nothing images: 2646
total training event images: 483
--
total validation nothing images: 400
total validation event images: 50
--
Total training images: 3129
Total validation images: 450
Found 3129 images belonging to 2 classes.
Found 450 images belonging to 2 classes.


In [ ]:
#Podme to navazit 

weight_for_0 = (1 / 483)*(3129)/2.0 
weight_for_1 = (1 / 2646)*(3129)/2.0

print("weight_for_0 = "+str(weight_for_0))
print("weight_for_1 = "+str(weight_for_1))

class_weight = {0: weight_for_0, 1: weight_for_1}

weight_for_0 = 3.239130434782609
weight_for_1 = 0.5912698412698413


**Praca na modeli**

In [ ]:
# Načítanie Cerberus modelu

tleia_model_cerberus = tf.keras.models.load_model(path+"/tleia_model_cerberus.h5")

"""
# Vytvorenie modelu Huge

tleia_model_cerberus = Sequential([
    Conv2D(16, 3, padding='same', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3), activation='relu'),
    MaxPooling2D(),
    Dropout(0.05),

    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.05),

    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.05),

    Conv2D(128, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.05),

    Flatten(),

    Dense(1024),
    Dropout(0.05),

    Dense(1024, activation='relu'),
    Dropout(0.05),

    Dense(1)
    
    ])
"""

"\n# Vytvorenie modelu Huge\n\ntleia_model_cerberus = Sequential([\n    Conv2D(16, 3, padding='same', input_shape=(IMG_HEIGHT, IMG_WIDTH ,3), activation='relu'),\n    MaxPooling2D(),\n    Dropout(0.05),\n\n    Conv2D(32, 3, padding='same', activation='relu'),\n    MaxPooling2D(),\n    Dropout(0.05),\n\n    Conv2D(64, 3, padding='same', activation='relu'),\n    MaxPooling2D(),\n    Dropout(0.05),\n\n    Conv2D(128, 3, padding='same', activation='relu'),\n    MaxPooling2D(),\n    Dropout(0.05),\n\n    Flatten(),\n\n    Dense(1024),\n    Dropout(0.05),\n\n    Dense(1024, activation='relu'),\n    Dropout(0.05),\n\n    Dense(1)\n    \n    ])\n"

In [ ]:
# Skompilovanie modelu a jeho sumarizacia Huge

METRICS = [
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.TruePositives(name='TN'),
      keras.metrics.FalsePositives(name='FN'),
      keras.metrics.TrueNegatives(name='TP'),
      keras.metrics.FalseNegatives(name='FP'), 
      #keras.metrics.Recall(name='recall'),
      #keras.metrics.AUC(name='auc'),
      #keras.metrics.SpecificityAtSensitivity(0.25, name="SpecATSen")
      ]

tleia_model_cerberus.compile(optimizer='adam',   #Podla porovnani najlepsi optimizer
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=METRICS) #['accuracy']

tleia_model_cerberus.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 200, 250, 16)      448       
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 100, 125, 16)      0         
_________________________________________________________________
dropout_39 (Dropout)         (None, 100, 125, 16)      0         
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 100, 125, 32)      4640      
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 50, 62, 32)        0         
_________________________________________________________________
dropout_40 (Dropout)         (None, 50, 62, 32)        0         
_________________________________________________________________
conv2d_27 (Conv2D)           (None, 50, 62, 64)       

In [ ]:
# Ucenie neuronovej siete Huge

history = tleia_model_cerberus.fit(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size,
    class_weight=class_weight)

Epoch 1/10
56/56 [==============================] - 309s 6s/step - loss: 0.0060 - accuracy: 0.0059 - precision: 0.9984 - TN: 149710.9062 - FN: 241.0000 - TP: 26140.4102 - FP: 1169.5000 - val_loss: 0.1839 - val_accuracy: 0.0059 - val_precision: 0.9984 - val_TN: 151201.5000 - val_FN: 241.7500 - val_TP: 26401.1250 - val_FP: 1177.1250
Epoch 2/10
56/56 [==============================] - 307s 5s/step - loss: 0.0050 - accuracy: 0.0059 - precision: 0.9984 - TN: 152679.5781 - FN: 243.9286 - TP: 26676.5723 - FP: 1181.3214 - val_loss: 0.1758 - val_accuracy: 0.0058 - val_precision: 0.9984 - val_TN: 154176.1250 - val_FN: 246.5000 - val_TP: 26925.7500 - val_FP: 1187.1250
Epoch 3/10
56/56 [==============================] - 305s 5s/step - loss: 0.0035 - accuracy: 0.0058 - precision: 0.9984 - TN: 155677.2188 - FN: 247.0000 - TP: 27177.2500 - FP: 1191.1428 - val_loss: 0.1867 - val_accuracy: 0.0058 - val_precision: 0.9984 - val_TN: 157159.7500 - val_FN: 248.1250 - val_TP: 27445.6250 - val_FP: 1196.0000
E

In [ ]:
# Ulozenie samotneho modeluHuge
 
tleia_model_cerberus.save(
    path+"/tleia_model_cerberus/", overwrite=True, 
    include_optimizer=True, save_format=True,
    signatures=None, options=None)

!pip install -q pyyaml h5py
tleia_model_cerberus.save(path+"/tleia_model_cerberus.h5")

INFO:tensorflow:Assets written to: /content/raw_data/My Drive/Space::Lab/ML_Left//tleia_model_cerberus/assets


Tuna zacina experimentovanie




```
#BEHEMOOOT model 

tleia_model_behemoth = tf.keras.applications.NASNetMobile(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), 
                                                          include_top=True, weights=None, 
                                                          input_tensor=None, pooling="avg", classes=1000)

METRICS = [
      keras.metrics.TruePositives(name='TN'),
      keras.metrics.FalsePositives(name='FN'),
      keras.metrics.TrueNegatives(name='TP'),
      keras.metrics.FalseNegatives(name='FP'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      #keras.metrics.SpecificityAtSensitivity(0.4, name="SpecATSen")
      ]
     
tleia_model_behemoth.compile(optimizer='adam',   
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=METRICS)

tleia_model_behemoth_sequential = Sequential()
tleia_model_behemoth_sequential.add(tleia_model_behemoth)
tleia_model_behemoth_sequential.add(Dense(1000, activation='tanh'))
tleia_model_behemoth_sequential.add(Dense(1, activation='relu'))

tleia_model_behemoth_sequential.compile(optimizer='adam',   
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=METRICS)

tleia_model_behemoth_sequential.summary()
------------------------
history = tleia_model_behemoth_sequential.fit(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size,
    class_weight=class_weight)
------------------------------
#Leviathan model  

tleia_model_leviathan = tf.keras.applications.Xception(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
    pooling=None,
    classes=1,
    classifier_activation="softmax",)

tleia_model_leviathan.compile(optimizer='adam',   
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=["accuracy"])

tleia_model_leviathan.summary()
--------------------------
history = tleia_model_leviathan.fit(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size,
    class_weight=class_weight)
---------------------------
#Hydra model 

#tleia_model_hydra = tf.keras.applications.NASNetLarge(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), include_top=True, weights=None, input_tensor=None, pooling=None, classes=1)

tleia_model_hydra.compile(optimizer='adam',   
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=['accuracy'])

tleia_model_hydra.summary()
--------------------------------
history = tleia_model_hydra.fit(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size 
    )
----------------------------------
#CERBERUS model 

tleia_model_cerberus = tf.keras.applications.VGG19(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
    pooling=None,
    classes=1,
    classifier_activation="softmax")

tleia_model_cerberus.compile(optimizer='adam',   
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=[tf.keras.metrics.Accuracy(),
                           tf.keras.metrics.Precision()])

tleia_model_cerberus.summary()
----------------------------
history = tleia_model_cerberus.fit(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=total_val // batch_size )
-----------------------------------
# Ulozenie oboch modelov (Behemoth)

# Behemoth  = NasNetMobile
# Leviathan = Xception
# Hydra     = NasNetLarge

tleia_model_behemoth.save(
    path+"/tleia_model_behemoth/", overwrite=True, include_optimizer=True, save_format=True,
    signatures=None, options=None)


tleia_model_behemoth.save(path+"/tleia_model_behemoth.h5")
--------------------------------
# Ulozenie oboch modelov (leviathan)

tleia_model_leviathan.save(
    path+"/tleia_model_leviathan/", overwrite=True, include_optimizer=True, save_format=True,
    signatures=None, options=None)

tleia_model_leviathan.save(path+"/tleia_model_leviathan.h5")
--------------------------------
# Ulozenie modelu Hydra

tleia_model_hydra.save(
    path+"/tleia_model_hydra/", overwrite=True, include_optimizer=True, save_format=True,
    signatures=None, options=None)

tleia_model_hydra.save(path+"/tleia_model_hydra.h5")
-----------------------------------
#Nacita modely

tleia_model_behemoth = tf.keras.models.load_model(path+"/tleia_model_behemoth/")
------------------------------
#Nacita modely

tleia_model_leviathan = tf.keras.models.load_model(path+"/tleia_model_leviathan/")
--------------------------------------------
#Nacita modely

tleia_model_hydra = tf.keras.models.load_model(path+"/tleia_model_hydra/")
---------------------------------------
```

